Cargamos las librerías, el conjunto de datos y algunas variables que emplearemos durante el proceso:

In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pandas_profiling
%matplotlib inline

df = pd.read_csv('./processed_data/houses_integrated.csv')

# Definimos las opciones de visualizacion
pd.set_option('display.max_columns', 500)

# Definimos un listado de colores para visualizaciones
clr = {'pr': '#5F66A1', 'yw': '#f3ca75',
       'mg': '#c874b9', 'gn': '#599d70', 'br': '#636261'}
b = '\033[94m'  # para texto azul
o = '\033[93m'  # para texto naranja
n = '\033[0m'   # para texto normal

# Omitimos los warnings
warnings.filterwarnings('ignore')

# Verificación de los datos

En esta fase verificaremos la calidad de los datos por medio de una serie de procesos.

## Descarte de las casas de alquiler

En el conjunto de datos que disponemos, se mezclan anuncios con casa en venta y alquiler. Este análisis se centra en la venta de casas, por lo tanto, los anuncios ligados al alquiler han de descartarse.

No obstante, no hay ningún campo que permita esta división de forma directa. Por ello, se opta por emplear el precio para inferir si una casa está a la venta o en alquiler, ya que, en principio, han de tener precios muy distintos.

Analizemos por lo tanto los precios por medio de un histograma, acotando los precios entre 2.500€ y 20.000€,  a fin de encontrar un umbral a partir del cual poder considerar que una casa está en venta y no en alquiler:

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 5), sharex=True)
sns.despine(left=True)

(df['price']
 .pipe(sns.distplot, hist=False, color=clr['pr'], kde_kws={"shade": True, 'clip': (2500, 20000)}))

plt.setp(axes, yticks=[])
plt.tight_layout()

Vemos que hasta los 8.000€ hay casas, y que luego no hay viviendas hasta cerca de los 15.000€. Asumiremos que 15.000€ es más probable que sea el precio de venta a que sea el precio por un alquiler mensual, y viceversa, pensaremos que hasta los 8.000€ son casas de alquiler. Así, se opta por descartar todas las casas por debajo de los 12.500€:

In [ ]:
h_num = df[df.price < 12500].shape[0]
df = df[df.price > 12500]
print(f'Se han descartado {h_num} casas de alquiler del dataset')

## Verificación de casas duplicadas

Puede que haya casas duplicadas en nuestro conjunto de datos. Cada anuncio de casa se define mediante un identificador único. Por ello, la forma más sencilla para verificar que no haya 2 anuncios repetidos es comprobando ese identificador:

In [ ]:
h_num = df[df.house_id.duplicated()].shape[0]
df.drop_duplicates(subset='house_id', inplace=True)
print(f'Se han descartado {h_num} casas duplicadas del dataset')

# Estandarización de variables

En este apartado se procede a estandarizar las variables, corrigiendo su tipo, y normalizando sus valores, (entendiendo por normalizar, asignarles el valor más representativo).

*NOTA: En este apartado no se tratarán ni los missing values ni los outliers. Su gestión se deja para más adelante.*

## Estudio de variables a estandarizar

Empezaremos analizando cómo han sido asignadas las variables:

In [ ]:
df.dtypes

De la lista superior, extraemos qué variables estudiaremos en la estandarización. En concreto, nos centraremos en 4 grupos: 
+ Por un lado estandarizaremos las variables numéricas que no hayan asignado como tal
+ Por otro lado, estudiaremos las variables categóricas
+ A continuación estudiaremos las variables categoricas binarias, es decir, booleanas
+ Finalmente, nos centraremos en las variables de texto ligadas a la ubicación

<br />

**VARIABLES NUMÉRICAS NO ASIGNADAS COMO TAL**

+ <font color=#5F66A1>*bath_num*<font>
+ <font color=#5F66A1>*room_num*<font>
+ <font color=#5F66A1>*garage*<font>


**VARIABLES CATEGÓRICAS**

+ <font color=#5F66A1>*condition*<font>
+ <font color=#5F66A1>*energetic_certif*<font>
+ <font color=#5F66A1>*floor*<font>
+ <font color=#5F66A1>*heating*<font>
+ <font color=#5F66A1>*house_type*<font>
+ <font color=#5F66A1>*orientation*<font>


**VARIABLES BOOLEANAS**

+ <font color=#5F66A1>*air_conditioner*<font>
+ <font color=#5F66A1>*balcony*<font>
+ <font color=#5F66A1>*built_in_wardrobe*<font>
+ <font color=#5F66A1>*chimney*<font>
+ <font color=#5F66A1>*garden*<font>
+ <font color=#5F66A1>*kitchen*<font>
+ <font color=#5F66A1>*lift*<font>
+ <font color=#5F66A1>*reduced_mobility*<font>
+ <font color=#5F66A1>*storage_room*<font>
+ <font color=#5F66A1>*swimming_pool*<font>
+ <font color=#5F66A1>*terrace*<font>
+ <font color=#5F66A1>*unfurnished*<font>


**VARIABLES DE TEXTO LIGADAS A LA UBICACIÓN**

+ <font color=#5F66A1>*loc_zone*<font>
+ <font color=#5F66A1>*loc_district*<font>
+ <font color=#5F66A1>*loc_city*<font>
+ <font color=#5F66A1>*loc_neigh*<font>

## Estandarización de variables numéricas

### <font color=#5F66A1>bath_num</font>

In [ ]:
print(df.bath_num.unique())

Vemos como la ausencia de baños se identifica como 'sin baños', en forma de texto. Lo corregimos, y definimos la variable como numérica:

In [ ]:
df['bath_num'] = (df['bath_num']
                  .str.replace('sin baños','0')
                  .astype('int64')
                 )

### <font color=#5F66A1>room_num</font>

In [ ]:
print(df.room_num.unique())

Aquí tambien la ausencia de habitaciones se identifica en forma de texto, como 'sin habitación'. Lo corregimos, y definimos la variable como numérica:

In [ ]:
df['room_num'] = (df['room_num']
                   .str.replace('sin habitación','0')
                   .astype('int64')
                  )

### <font color=#5F66A1>garage</font>

In [ ]:
print(df.garage.unique()[:15])

Vemos que la variable refleja el precio de la plaza de garaje en forma de texto. Por lo tanto, extraemos el precio del texto y establecemos que la variable sea numérica:

In [ ]:
df['garage'] = (df['garage']
                .str.replace('plaza de garaje incluida en el precio','0')
                .str.replace('.','')
                .str.extract('(\d{1,4})')
                .astype('float64')
               )

## Estandarización de variables categóricas

### <font color=#5F66A1>condition</font>

In [ ]:
print(df.condition.unique())

Vemos que hay 3 posibles categorías. Redefinimos los nombres de las categorías para mejorar su interpretabilidad.

In [ ]:
df = df.replace({'condition':
                 {
                  'segunda mano/buen estado':'buen estado',
                  'segunda mano/para reformar':'para reformar',
                  'promoción de obra nueva':'obra nueva'
                 }})

df['condition'] = df['condition'].astype('category')

### <font color=#5F66A1>energetic_certif</font>

In [ ]:
print(df.energetic_certif.unique())

No requiere ningun tratamiento salvo definirla como categórica:

In [ ]:
df['energetic_certif'] = df['energetic_certif'].astype('category')

### <font color=#5F66A1>floor</font>

In [ ]:
print(df.floor.unique()[:15])

Vemos que, a grandes rasgos, esta variable esconde lo que podrían ser 3 variables distintas:

+ Número de planta en la que se encuentra el domicilio
+ Indicador de si la planta está en el interior o en el exterior
+ Número de plantas de las que consta el domicilio

Por lo tanto, se procede a dividir la variable en 3. 

<br />

Empezamos generando la variable booleana <font color=#5F66A1>__indoor__</font>, que tomará el valor True si el domicilio está en el interior, y False en el caso contrario:

In [ ]:
df['indoor'] = (df['floor']
                .str.contains('interior')
                .astype('category')   # no se define como bool porque aún contiene NaN
               )

print(df['indoor'].unique())

<br />

A continuación crearemos la variable <font color=#5F66A1>__floor_count__</font>, que recogerá el número de plantas de las que consta la casa.

Para ello, empezamos asignando como valor 1 a todas las casas que tengan información sobre la planta (las que estan como NaN las dejaremos por ahora tal cual, puesto que no tenemos información al respecto)

In [ ]:
df['floor_count'] = np.nan
df['floor_count'][df.floor.notna()] = 1

A continuación, añadiremos la información sobre las casas con más de una planta. Para ello, seleccionaremos todas las celdas que contengan la palabra 'plantas' (en plural), y de allí extraeremos el número por medio de una expresión regular:

In [ ]:
f_c = (df['floor'][df.floor.str.contains('plantas', na=False)]
       .str.extract('(\d+)')
       .astype('float64')
       .rename(columns={0:'floor_count'})
      )

Una vez hemos extraído el número de plantas de las casas con más de una, actualizamos la variable:

In [ ]:
df.update(f_c)
print(df.floor_count.unique())

<br />

Por último modificaremos la variable <font color=#5F66A1>__floor__</font>, para que solo recoja el número de planta en el que se encuentra el piso:

In [ ]:
df['floor'] = (df['floor']
               .str.replace('exterior', '')
               .str.replace('interior', '')
               .str.replace('1 planta', '')
               .str.replace('(\d+) plantas', '')
               .str.strip()
               .str.replace('planta ', '')
               .str.replace('-1', '-1ª')
               .str.replace('-2', '-2ª')
               .replace('', np.nan)
               .astype('category')
              )

print(df.floor.unique())

### <font color=#5F66A1>heating</font>

In [ ]:
print(df.heating.unique())

Sustraemos la palabra 'calefacción' para mayor interpretabilidad, y la definimos como categórica:

In [ ]:
df['heating'] = (df['heating']
                 .str.replace('calefacción ', '')
                 .astype('category')
                )

### <font color=#5F66A1>house_type</font>

In [ ]:
print(df.house_type.unique())

Eliminamos los espacios en blanco y la definimos como categórica:

In [ ]:
df['house_type'] = (df['house_type']
                    .str.strip()
                    .astype('category')
                   )

### <font color=#5F66A1>orientation</font>

In [ ]:
print(df.orientation.unique())

No requiere estandarización salvo definirla como categórica:

In [ ]:
df['orientation'] = (df['orientation'].astype('category'))

## Estandarización de variables booleanas

Analizamos qué valores tienen las variables booleanas:

In [ ]:
print(f'Valores para {b}air_conditioner{n}: {df.air_conditioner.unique()}')
print(f'Valores para {b}balcony{n}: {df.balcony.unique()}')
print(f'Valores para {b}built_in_wardrobe{n}: {df.built_in_wardrobe.unique()}')
print(f'Valores para {b}chimney{n}: {df.chimney.unique()}')
print(f'Valores para {b}garden{n}: {df.garden.unique()}')
print(f'Valores para {b}kitchen{n}: {df.kitchen.unique()}')
print(f'Valores para {b}lift{n}: {df.lift.unique()}')
print(f'Valores para {b}reduced_mobility{n}: {df.reduced_mobility.unique()}')
print(f'Valores para {b}storage_room{n}: {df.storage_room.unique()}')
print(f'Valores para {b}swimming_pool{n}: {df.swimming_pool.unique()}')
print(f'Valores para {b}terrace{n}: {df.terrace.unique()}')
print(f'Valores para {b}unfurnished{n}: {df.unfurnished.unique()}')

Por un lado  vemosque las variables <font color=#5F66A1>__kitchen__</font> y <font color=#5F66A1>__unfurnished__</font> solo contienen *NaN*, probablemente porque se traten de variables ligadas exclusivamente a casas de alquiler. Por lo tanto, las eliminamos:

In [ ]:
df = df.drop(columns=['kitchen', 'unfurnished'])

Por otra parte, vemos que la variable <font color=#5F66A1>__lift__</font>, a pesar de ser booleana, contiene valores NaN, por lo que la definimos como categórica por ahora:

In [ ]:
df['lift'] = df['lift'].astype('category')

Finalmente, el resto de variables categóricas las definimos como booleanas:

In [ ]:
df['air_conditioner'] = df['air_conditioner'].astype('bool')
df['balcony'] = df['balcony'].astype('bool')
df['built_in_wardrobe'] = df['built_in_wardrobe'].astype('bool')
df['chimney'] = df['chimney'].astype('bool')
df['garden'] = df['garden'].astype('bool')
df['reduced_mobility'] = df['reduced_mobility'].astype('bool')
df['storage_room'] = df['storage_room'].astype('bool')
df['swimming_pool'] = df['swimming_pool'].astype('bool')
df['terrace'] = df['terrace'].astype('bool')

In [ ]:
print(f'Nuevos valores para {b}air_conditioner{n}: {df.air_conditioner.unique()}')
print(f'Nuevos valores para {b}balcony{n}: {df.balcony.unique()}')
print(f'Nuevos valores para {b}built_in_wardrobe{n}: {df.built_in_wardrobe.unique()}')
print(f'Nuevos valores para {b}chimney{n}: {df.chimney.unique()}')
print(f'Nuevos valores para {b}garden{n}: {df.garden.unique()}')
print(f'Nuevos valores para {b}lift{n}: {df.lift.unique()}')
print(f'Nuevos valores para {b}reduced_mobility{n}: {df.reduced_mobility.unique()}')
print(f'Nuevos valores para {b}storage_room{n}: {df.storage_room.unique()}')
print(f'Nuevos valores para {b}swimming_pool{n}: {df.swimming_pool.unique()}')
print(f'Nuevos valores para {b}terrace{n}: {df.terrace.unique()}')

## Estandarización de variables ligadas a la ubicación

La localización de cada vivienda viene especificada por medio de 4 variables distintas, las cuales están organizadas de forma jerárquica del siguiente modo:

+ <font color=#5F66A1>__loc_zone__</font> &nbsp; > &nbsp;<font color=#5F66A1>__loc_city__</font> &nbsp;> &nbsp;<font color=#5F66A1>__loc_district__</font> &nbsp;> &nbsp;<font color=#5F66A1>__loc_neigh__</font>

### <font color=#5F66A1>loc_zone</font>

In [ ]:
print(df.loc_zone.unique())

Definimos las provincias con las iniciales y convertimos la variable en categórica:

In [ ]:
df['loc_zone'] = (df['loc_zone']
                  .str.replace(', Vizcaya',' (BIZ)')
                  .str.replace(', Guipúzcoa',' (GIP)')
                  .astype('category')
                 )

print(df.loc_zone.unique())

### <font color=#5F66A1>loc_city</font>

In [ ]:
print(df.loc_city.unique()[:15])

Los valores están ya lo suficientemente estandarizados, por lo que los dejamos como están, y establecemos la variable como categórica:

In [ ]:
df['loc_city'] = df['loc_city'].astype('category')

### <font color=#5F66A1>loc_district</font>

In [ ]:
print(f"Número de distritos: {len(df.loc_district.unique())}")

Vemos que hay {{len(df.loc_district.unique())}} distritos distintos. No obstante, en muchos casos, en vez del distrito como tal, el campo guarda información relativa a la calle, urbanización, etc., como puede verse a continuación:

In [ ]:
print(df.sample(frac=1, random_state=89)['loc_district'][df.loc_district.notnull()].head(5))

Dado que cada distrito está precedido por la palabra 'Distrito', eliminamos todos los valores que no coincidan con este patrón:

In [ ]:
df['loc_district'] = (df['loc_district']
                      .str.extract(r'(Distrito .*)')[0]
                      .str.replace('Distrito ','')
                     )

Por otro lado, para asegurarnos de que no haya distritos distintos compartiendo un mismo nombre, agregaremos las iniciales de cada ciudad:

In [ ]:
df['loc_district'] = (np.where(df.loc_district.notnull()
                      , df.loc_district+' ('+df.loc_city.str[:4]+')'
                      , np.nan)
                     )

print(df.sample(frac=1, random_state=89)['loc_district'][df.loc_district.notnull()].head(5))

Finalmente, comprobamos el número de distritos que hay tras el proceso de limpieza:

In [ ]:
print(f"Número de distritos tras limpieza: {len(df.loc_district.unique())}")

### <font color=#5F66A1>loc_neigh</font>

In [ ]:
print(f"Número de barrios: {len(df.loc_neigh.unique())}")

Vemos que hay {{len(df.loc_neigh.unique())}} barrios distintos. Sin embargo, sucede lo mismo que sucedía con el distrito, que en vez del barrio, en muchos casos el campo guarda información relativa a la calle, urbanización, etc.:

In [ ]:
print(df.sample(frac=1, random_state=89)['loc_neigh'][df.loc_neigh.notnull()].head(5))

En este caso, el barrio viene precedido siempre por la palabra 'Barrio'. Así, repetimos el mismo proceso que en el anterior caso:

In [ ]:
df['loc_neigh'] = (df['loc_neigh']
                      .str.extract(r'(Barrio .*)')[0]
                      .str.replace('Barrio ','')
                     )

df['loc_neigh'] = (np.where(df.loc_neigh.notnull()
                      , df.loc_neigh+' ('+df.loc_city.str[:4]+')'
                      , np.nan)
                  )

print(df.sample(frac=1, random_state=89)['loc_neigh'][df.loc_neigh.notnull()].head(5))

Finalmente, comprobamos el número de barrios que hay tras el proceso de limpieza:

In [ ]:
print(f"Número de barrios tras limpieza: {len(df.loc_neigh.unique())}")

# Gestión de ruido y valores extremos

En este apartado cotejaremos que los valores estén dentro de un rango plausible. De este modo, por cada variable numérica, gestionaremos tanto los valores extremos como las incongruencias. Además, se cotejará también la veracidad de algunas variables categóricas. 

## Ruido y outliers en variables numéricas

### <font color=#5F66A1>m2_real</font> & <font color=#5F66A1>m2_useful</font>

Empezemos mostrando en un boxplot la dispersión de los metros cuadadros reales de las viviendas:

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'m2_real']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Observamos que hay 2 casos muy extremos. Los exploramos a fin de determinar cómo tratarlos:

In [ ]:
df.query('m2_real > 50000').style.set_properties(subset=['ad_description'], **{'min-width': '400px'})

En la descripción del anuncio se ve que en un caso han sumado 3 ceros de más a la superficie real, y en el otro, han asignado el valor de la párcela. Por lo tanto, dado que en la descripción se detalla el tamaño real, se corrigen manualmente, y se vuelve a representar el boxplot:

In [ ]:
df['m2_real'][df.house_id == 82667064] = 415
df['m2_real'][df.house_id == 39173329] = 300

f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'m2_real']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Pese a haber resuelto los casos más extremos, vemos que aún siguen existiendo valores muy alejados del resto. De modo que volvemos a analizarlos a fin de determinar qué tratamiento darles:

In [ ]:
out_m2 = len(df[df['m2_real'] >1500])
(df
 .query('m2_real > 1500')
 .sort_values('m2_real',ascending=False)
 .style.set_properties(subset=['ad_description'], **{'min-width': '1000px'})
)

Observamos que hay {{out_m2}} viviendas con precios superiores a los 1.500 $m^2$ reales, entre las cuales hay un convento, un camping, una nave industrial... este tipo de edificios quedan fuera de nuestro objeto de estudio y podrían generar distorsión, por lo que se eliminan. Por otro lado, encontramos nuevamente viviendas que tienen mal definida su superficie; éstas las corregimos de forma manual mediante la información contenida en otros campos:

In [ ]:
df['m2_real'][df.house_id == 82012713] = 90
df['m2_real'][df.house_id == 39733981] = 46

h_num = len(df[df.m2_real > 1500])
df = df[df.m2_real < 1500]
print(f'Se han descartado {h_num} viviendas')

Por otro lado, en el otro extremo, exploramos si hay viviendas con valores de superficie nulos o negativos: 

In [ ]:
cnt = len(df.query('m2_real < 1'))
print(f"Hay {cnt} viviendas con superficies nulas o negativas")

Una vez hemos gestionado los valores extremos de la variable <font color=#5F66A1>__m2_real__</font>, hacemos lo propio con <font color=#5F66A1>__m2_useful__</font>:

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'m2_useful']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Vemos que el valor más extremo se encuentra en los 1.200 $m^2$, lo cual entra dentro del rango plausible tras haber eliminado las casas con más de 1.500 $m^2$ reales, por lo que en este caso, no se realiza ninguna acción más, y se procede a verificar que no haya valores nulos o negativos:

In [ ]:
cnt = len(df.query('m2_useful < 1'))
print(f"Hay {cnt} viviendas con superficies nulas o negativas")

Por último, los metros útiles de una vivienda han de ser inferiores a sus metros reales. Verificamos si hay algún caso que no cumpla este criterio:

In [ ]:
cnt = len(df.query('m2_real < m2_useful'))
print(f"Hay {cnt} viviendas con incongruencias entre las superficies")

Además, por lo general, los metros útiles se situan en torno a un 0%-20% por debajo de los metros reales. Por ello, exploremos gráficamente esta relación para hallar valores extremos:

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(12, 6), sharex=True)
sns.despine(left=True)

(df.pipe((sns.scatterplot, 'data'),x="m2_real", y="m2_useful", color=clr['pr'], legend="full"))

plt.tight_layout()

# añadir 2 rectas... una en y=x (0%), la otra en y=x/0.8 (20%)

Comprobamos que, en efecto, la mayoría de viviendas disponen la relación esperada. No obstante, también encontramos viviendas con muy pocos metros útiles para los metros reales que tienen. Para cotejar si son plausibles, exploremos las 5 viviendas con mayor diferencia entre ambas superficies:

In [ ]:
df_out = df
df_out['m2_relation'] = df['m2_real'] - df['m2_useful']

(df_out
    .sort_values('m2_relation', ascending=False)[:5]
    .style.set_properties(subset=['ad_description'], **{'min-width': '1400px'})
)

Salvo el caso más extremo, a la cual se le ha asigando un 0 de menos, el resto de viviendas no presentan anomalías aparentes. Por lo tanto, se modifica el valor más extremo, y se asume que el resto entran dentro de un rango plausible:

In [ ]:
df['m2_useful'][df.house_id == 85089573] = 740

### <font color=#5F66A1>price</font>

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(12, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'price']
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Vemos que los datos tienen una distribución asimétrica positiva, con una larga cola que se extiende hacia la derecha.

Los valores más extremos corresponden a casas que cuestan entre 3 y 4 millones, las cuales son plausibles, por lo que se opta por mantenerlos.

### <font color=#5F66A1>construct date</font>

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'construct_date']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

Vemos que hay ...

In [ ]:
del_count = len(df.query('construct_date > 2030'))
df['construct_date'] = (np.where(df.construct_date > 2030
                      , np.nan
                      , df.construct_date)
                        )
print(f"Se han modificado {del_count} valores")

### <font color=#5F66A1>garage</font>

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'garage']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

### <font color=#5F66A1>room_num</font>

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
sns.despine(left=True)

(df
 .loc[:,'room_num']
 .dropna()
 .pipe(sns.boxplot, color=clr['pr'], boxprops=dict(alpha=.7)))

plt.setp(axes, yticks=[])
plt.tight_layout()

In [ ]:
df[df.floor == '24ª']

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(15, 5), sharex=True)

(df
 .pipe((sns.catplot, 'data'), ax=axes, x='floor', kind="count"))

# Resolver datos incompletos

Empezamos analizando como estan repartidos nuestros *missing values*:

In [ ]:


for col in df.columns:
    na_count = sum(pd.isna(df[col]))
    na_perc = round((na_count*100)/len(df),1)
    if(na_count > 0 and na_perc < 50):
        print(f"- {b}{col}{n}: {na_perc}%")
    elif(na_count > 0 and na_perc >= 50):
        print(f"- {b}{col}{n}: {o}{na_perc}%{n}")

Vemos que hay muchas variables con *missing values*, algunas incluso con más del 50% de sus valores. Trataremos todas, excepto la descripción de los anuncios, ya que debido a que solo la usaremos como ayuda en la fase exploratoria, se dejará tal cual.

In [ ]:
pandas_profiling.ProfileReport(df)

In [ ]:
#(msno.nullity_sort(df, sort="ascending").pipe(msno.bar, color=(0.1,0.3,0.3)));

In [ ]:
(df.query('price > 3000000')
 .style.set_properties(subset=['ad_description'], **{'min-width': '1000px'})
);   

In [ ]:
(df
 .drop(columns=['house_id'])
 .describe(percentiles=[])
 .loc[['min','max'],:]
)